In [2]:
import pandas as pd
import numpy as np
import warnings
import geocoder

import matplotlib.pyplot as plt
import seaborn as sns

from mock import train_val_test, xy_split, scaled_data, clean_and_converts
from mock2 import eval_model, train_model
from env import get_connection

from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.model_selection import GridSearchCV

In [3]:
# Call the clean_and_convert() function to obtain a cleaned and converted DataFrame.
df = clean_and_converts()

# Display the first few rows of the DataFrame 'df' to inspect the data.
df.shape

(52441, 22)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   finishedsquarefeet12    52194 non-null  float64 
 1   squarefeet              52359 non-null  float64 
 2   latitude                52441 non-null  float64 
 3   regionidzip             52415 non-null  float64 
 4   longitude               52441 non-null  float64 
 5   lotsizesquarefeet       52072 non-null  float64 
 6   logerror                52441 non-null  float64 
 7   year_built              52325 non-null  float64 
 8   id                      52441 non-null  int64   
 9   id                      52441 non-null  int64   
 10  rawcensustractandblock  52441 non-null  float64 
 11  regionidcity            51404 non-null  float64 
 12  parcelid                52441 non-null  int64   
 13  parcelid                52441 non-null  int64   
 14  bathrooms             

In [3]:
df.head()

,finishedsquarefeet12,squarefeet,latitude,regionidzip,longitude,lotsizesquarefeet,logerror,year_built,id,id,...,parcelid,parcelid,bathrooms,bedrooms,fips,tax_value,bedrooms_bin,bathrooms_bin,squarefeet_bin,decades
0,3100.0,3100.0,33634931.0,96978.0,-117869207.0,4506.0,0.025595,1998.0,1727539,0,...,14297519,14297519,3.5,4.0,6059.0,1023282.0,6,4,3500,2000
1,1465.0,1465.0,34449266.0,97099.0,-119281531.0,12647.0,0.055619,1967.0,1387261,1,...,17052889,17052889,1.0,2.0,6111.0,464000.0,3,2,1500,1970
2,1243.0,1243.0,33886168.0,97078.0,-117823170.0,8432.0,0.005383,1962.0,11677,2,...,14186244,14186244,2.0,3.0,6059.0,564778.0,4,3,1500,1970
3,2376.0,2376.0,34245180.0,96330.0,-118240722.0,13038.0,-0.103410,1970.0,2288172,3,...,12177905,12177905,3.0,4.0,6037.0,145143.0,6,4,2500,1980
4,2962.0,2962.0,34145202.0,96293.0,-118179824.0,63000.0,-0.001011,1950.0,781532,6,...,12095076,12095076,3.0,4.0,6037.0,773303.0,6,4,3000,1960


In [4]:
df.isna().sum().head(10)

finishedsquarefeet12    247
squarefeet               82
latitude                  0
regionidzip              26
longitude                 0
lotsizesquarefeet       369
logerror                  0
year_built              116
id                        0
id                        0
dtype: int64

columns_to_drop = ['tax_amount', 'basementsqft', 'airconditioningtypeid', 'architecturalstyletypeid', 'censustractandblock', 'buildingclasstypeid', 'buildingqualitytypeid', 'decktypeid', 'finishedfloor1squarefeet', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fireplacecnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'heatingorsystemtypeid', 'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'propertyzoningdesc', 'taxdelinquencyflag', 'taxdelinquencyyear', 'numberofstories', 'fireplaceflag', 'yardbuildingsqft26', 'yardbuildingsqft17', 'unitcnt', 'typeconstructiontypeid', 'threequarterbathnbr', 'storytypeid', 'regionidneighborhood', 'landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'transactiondate', 'propertycountylandusecode']
df.drop(columns=columns_to_drop, inplace=True)

finishedsquarefeet12 
squarefeet
latitude
regionidzip 
longitude
lotsizesquarefeet 
logerror 
year_built
id 
id
rawcensustractandblock  
regionidcity  
parcelid   
parcelid    
bathrooms   
decades   
bedrooms    

In [5]:
df.shape

(52441, 22)

In [6]:
df.isna().sum().head(24)

finishedsquarefeet12       247
squarefeet                  82
latitude                     0
regionidzip                 26
longitude                    0
lotsizesquarefeet          369
logerror                     0
year_built                 116
id                           0
id                           0
rawcensustractandblock       0
regionidcity              1037
parcelid                     0
parcelid                     0
bathrooms                    0
bedrooms                     0
fips                         0
tax_value                    1
bedrooms_bin                 0
bathrooms_bin                0
squarefeet_bin             864
decades                    170
dtype: int64

In [7]:
df = df.dropna()

In [8]:
df.shape

(50027, 22)

In [9]:
df.dtypes

finishedsquarefeet12       float64
squarefeet                 float64
latitude                   float64
regionidzip                float64
longitude                  float64
lotsizesquarefeet          float64
logerror                   float64
year_built                 float64
id                           int64
id                           int64
rawcensustractandblock     float64
regionidcity               float64
parcelid                     int64
parcelid                     int64
bathrooms                  float64
bedrooms                   float64
fips                       float64
tax_value                  float64
bedrooms_bin              category
bathrooms_bin             category
squarefeet_bin            category
decades                   category
dtype: object

In [10]:
df.head()

,finishedsquarefeet12,squarefeet,latitude,regionidzip,longitude,lotsizesquarefeet,logerror,year_built,id,id,...,parcelid,parcelid,bathrooms,bedrooms,fips,tax_value,bedrooms_bin,bathrooms_bin,squarefeet_bin,decades
0,3100.0,3100.0,33634931.0,96978.0,-117869207.0,4506.0,0.025595,1998.0,1727539,0,...,14297519,14297519,3.5,4.0,6059.0,1023282.0,6,4,3500,2000
1,1465.0,1465.0,34449266.0,97099.0,-119281531.0,12647.0,0.055619,1967.0,1387261,1,...,17052889,17052889,1.0,2.0,6111.0,464000.0,3,2,1500,1970
2,1243.0,1243.0,33886168.0,97078.0,-117823170.0,8432.0,0.005383,1962.0,11677,2,...,14186244,14186244,2.0,3.0,6059.0,564778.0,4,3,1500,1970
3,2376.0,2376.0,34245180.0,96330.0,-118240722.0,13038.0,-0.103410,1970.0,2288172,3,...,12177905,12177905,3.0,4.0,6037.0,145143.0,6,4,2500,1980
4,2962.0,2962.0,34145202.0,96293.0,-118179824.0,63000.0,-0.001011,1950.0,781532,6,...,12095076,12095076,3.0,4.0,6037.0,773303.0,6,4,3000,1960


In [11]:
df['state'] = 'CA'

In [12]:
# Define a dictionary to map FIPS codes to county names
fips_to_county = {
    6037: 'Los Angeles County',
    6059: 'Orange County',
    6111: 'Ventura County'
}

# Create the 'county' column by mapping the FIPS codes
df['county'] = df['fips'].replace(fips_to_county)

In [13]:
# Use pd.get_dummies to convert the columns into dummies
df = pd.get_dummies(df, columns=['county', 'state'])

In [14]:
df.head()

,finishedsquarefeet12,squarefeet,latitude,regionidzip,longitude,lotsizesquarefeet,logerror,year_built,id,id,...,fips,tax_value,bedrooms_bin,bathrooms_bin,squarefeet_bin,decades,county_Los Angeles County,county_Orange County,county_Ventura County,state_CA
0,3100.0,3100.0,33634931.0,96978.0,-117869207.0,4506.0,0.025595,1998.0,1727539,0,...,6059.0,1023282.0,6,4,3500,2000,False,True,False,True
1,1465.0,1465.0,34449266.0,97099.0,-119281531.0,12647.0,0.055619,1967.0,1387261,1,...,6111.0,464000.0,3,2,1500,1970,False,False,True,True
2,1243.0,1243.0,33886168.0,97078.0,-117823170.0,8432.0,0.005383,1962.0,11677,2,...,6059.0,564778.0,4,3,1500,1970,False,True,False,True
3,2376.0,2376.0,34245180.0,96330.0,-118240722.0,13038.0,-0.103410,1970.0,2288172,3,...,6037.0,145143.0,6,4,2500,1980,True,False,False,True
4,2962.0,2962.0,34145202.0,96293.0,-118179824.0,63000.0,-0.001011,1950.0,781532,6,...,6037.0,773303.0,6,4,3000,1960,True,False,False,True


In [15]:
# Split the DataFrame 'df' into training, validation, and test sets using the train_val_test function.
train, val, test = train_val_test(df)

In [16]:
# Check the dimensions (number of rows and columns) of the training, validation, and test sets.
train.shape, val.shape, test.shape

((35018, 26), (7504, 26), (7505, 26))

In [17]:
# Define a list of column names to be scaled, excluding the 'tax_value' column.
to_scale = df.drop(columns=['tax_value']).columns

# Scale the training, validation, and test datasets using the specified columns.
train, val, test = scaled_data(train, val, test, to_scale, scaler_type='minmax')

train.head()

,finishedsquarefeet12,squarefeet,latitude,regionidzip,longitude,lotsizesquarefeet,logerror,year_built,id,id,...,fips,tax_value,bedrooms_bin,bathrooms_bin,squarefeet_bin,decades,county_Los Angeles County,county_Orange County,county_Ventura County,state_CA
12650,0.501129,0.501129,0.546056,0.001498,0.588121,0.001693,0.572231,0.356522,0.885777,0.242572,...,0.000000,1319825.0,0.043478,0.142857,0.555556,0.363636,1.0,0.0,0.0,0.0
24542,0.279614,0.279614,0.431156,0.001679,0.835087,0.000917,0.595267,0.556522,0.683202,0.470081,...,0.000000,133899.0,0.086957,0.142857,0.222222,0.545455,1.0,0.0,0.0,0.0
35215,0.240608,0.240608,0.357690,0.003576,0.844042,0.000079,0.591695,0.747826,0.719068,0.670519,...,0.297297,197803.0,0.043478,0.142857,0.222222,0.727273,0.0,1.0,0.0,0.0
20857,0.274482,0.274482,0.592220,0.001413,0.545306,0.000897,0.593262,0.478261,0.995675,0.400657,...,0.000000,449000.0,0.086957,0.142857,0.222222,0.454545,1.0,0.0,0.0,0.0
42700,0.239786,0.239786,0.272867,0.003405,0.873301,0.000235,0.608868,0.547826,0.719256,0.813104,...,0.297297,56379.0,0.086957,0.142857,0.222222,0.545455,0.0,1.0,0.0,0.0


In [18]:
# Split the 'train' dataset into feature matrix (X_train) and target vector (y_train).
X_train, y_train = xy_split(train)

# Split the 'val' dataset into feature matrix (X_val) and target vector (y_val).
X_val, y_val = xy_split(val)

In [19]:
# Calculate the mean and median of the target vector 'y_train'.
y_train.mean(), y_train.median()

(482910.5436632589, 364341.5)

In [20]:
# Create a DataFrame 'baselines' with three columns: 'y_actual,' 'y_mean,' and 'y_median.'
baselines = pd.DataFrame({
    'y_actual': y_train,        # Actual target values from the training dataset.
    'y_mean': y_train.mean(),  # Mean of the target values from the training dataset.
    'y_median': y_train.median()  # Median of the target values from the training dataset.
})


baselines.head()

,y_actual,y_mean,y_median
12650,1319825.0,482910.543663,364341.5
24542,133899.0,482910.543663,364341.5
35215,197803.0,482910.543663,364341.5
20857,449000.0,482910.543663,364341.5
42700,56379.0,482910.543663,364341.5


In [21]:
# Evaluate the model's performance by comparing its predictions against the mean ('y_mean') of the target values.
eval_model(baselines.y_actual, baselines.y_mean)

523729.6325773532

In [22]:
# Evaluate the model's performance by comparing its predictions against the median ('y_median') of the target values.
eval_model(baselines.y_actual, baselines.y_median)

536983.5622761921

## LinearRegression

In [23]:
# Create an instance of the Linear Regression model.
lm = LinearRegression()

# Train the Linear Regression model on the training data (X_train, y_train).
# Evaluate its performance using the validation data (X_val, y_val).
train_model(lm, X_train, y_train, X_val, y_val)

The train RMSE is 422640.7368114309.
The validate RMSE is 413432.47373416397.


LinearRegression()

In [24]:
def select_kbest(X, y, k):
   
    # Create an instance of SelectKBest with the f_regression scoring function and select the top 2 features.
    skb = SelectKBest(score_func=f_regression, k=k)
    
    # Fit the SelectKBest instance to your feature variables (X) and target variable (y).
    skb.fit(X, y)

    # Get a Boolean mask indicating the selected features by SelectKBest.
    skb_mask = skb.get_support(indices=True)
    
    return pd.DataFrame(X.columns[skb_mask])

In [25]:
select_kbest(X_train, y_train, k=5)

,0
0,finishedsquarefeet12
1,squarefeet
2,bathrooms
3,bathrooms_bin
4,squarefeet_bin


## LassoLars

In [26]:
# Create an instance of the LassoLars regression model with alpha=0.5.
ll = LassoLars(alpha=0.1)

# Train the LassoLars model on the training data (X_train, y_train).
# Evaluate its performance using the validation data (X_val, y_val).
train_model(ll, X_train, y_train, X_val, y_val)

The train RMSE is 440586.5674935113.
The validate RMSE is 430577.7126510773.


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_least_angle.py:645: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=1.339e+04, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_least_angle.py:645: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 4 iterations, i.e. alpha=4.837e+03, with an active set of 4 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_least_angle.py:645: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 6 iterations, i.e. alpha=4.138e+03, with an active set of 6 regressors, and the 

LassoLars(alpha=0.1)

## PolynomialFeatures

In [27]:
# Create an instance of PolynomialFeatures to perform feature expansion.
poly = PolynomialFeatures()

# Transform the training features (X_train) into polynomial features (X_train_s).
X_train_s = poly.fit_transform(X_train)

# Transform the validation features (X_val) into polynomial features (X_val_s).
X_val_s = poly.fit_transform(X_val)

In [28]:
# Comparing the dimensions (number of rows and columns) of X_train_s and X_train
X_train_s.shape, X_train.shape

((35018, 351), (35018, 25))

## LinearRegression with polynomial features

In [29]:
# Create an instance of the Linear Regression model.
lm = LinearRegression()

# Train the Linear Regression model on the training data with polynomial features (X_train_s, y_train).
# Evaluate its performance using the validation data with polynomial features (X_val_s, y_val).
train_model(lm, X_train_s, y_train, X_val_s, y_val)

The train RMSE is 386586.36779793457.
The validate RMSE is 383290.69481229817.


LinearRegression()

## TweedieRegressor

In [30]:
# Create an instance of the Tweedie Regressor model.
tweedie = TweedieRegressor()

# Train the Tweedie Regressor model on the training data with polynomial features (X_train_s, y_train).
# Evaluate its performance using the validation data with polynomial features (X_val_s, y_val).
train_model(tweedie, X_train_s, y_train, X_val_s, y_val)

The train RMSE is 473621.4000199033.
The validate RMSE is 468397.9644899115.


TweedieRegressor()

In [31]:
# Create an instance of the Tweedie Regressor model.
tweedie = TweedieRegressor(power=1, alpha=0.01, link='auto', max_iter=170)

# Train the Tweedie Regressor model on the training data with polynomial features (X_train_s, y_train).
# Evaluate its performance using the validation data with polynomial features (X_val_s, y_val).
train_model(tweedie, X_train_s, y_train, X_val_s, y_val)

The train RMSE is 392120.79512608214.
The validate RMSE is 383961.4681469777.


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_glm/glm.py:282: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


TweedieRegressor(alpha=0.01, max_iter=170, power=1)

## RandomForestRegressor

In [32]:
# Create an instance of the RandomForestRegressor model.
rf = RandomForestRegressor()

# Train the RandomForestRegressor model on the training data (X_train_s, y_train).
# Evaluate its performance using the validation data (X_val_s, y_val).
train_model(rf, X_train, y_train, X_val, y_val)

The train RMSE is 139224.02650986606.
The validate RMSE is 357930.1766435903.


RandomForestRegressor()

In [33]:
feature_importances = rf.feature_importances_

In [34]:
len(feature_importances)

25

In [35]:
X_train.columns

Index(['finishedsquarefeet12', 'squarefeet', 'latitude', 'regionidzip',
       'longitude', 'lotsizesquarefeet', 'logerror', 'year_built', 'id', 'id',
       'rawcensustractandblock', 'regionidcity', 'parcelid', 'parcelid',
       'bathrooms', 'bedrooms', 'fips', 'bedrooms_bin', 'bathrooms_bin',
       'squarefeet_bin', 'decades', 'county_Los Angeles County',
       'county_Orange County', 'county_Ventura County', 'state_CA'],
      dtype='object')

In [36]:
# Create a DataFrame to display feature names and their importances
importances_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

In [37]:
# Sort the DataFrame by importance in descending order
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [38]:
print(importances_df)

                      Feature  Importance
1                  squarefeet    0.170137
0        finishedsquarefeet12    0.158313
3                 regionidzip    0.090986
4                   longitude    0.088112
2                    latitude    0.087261
5           lotsizesquarefeet    0.067891
6                    logerror    0.055110
7                  year_built    0.046926
8                          id    0.040927
9                          id    0.038341
11               regionidcity    0.029965
10     rawcensustractandblock    0.028468
13                   parcelid    0.024719
12                   parcelid    0.024166
14                  bathrooms    0.015864
20                    decades    0.011426
15                   bedrooms    0.008543
17               bedrooms_bin    0.005988
18              bathrooms_bin    0.003731
19             squarefeet_bin    0.002035
21  county_Los Angeles County    0.000407
23      county_Ventura County    0.000402
16                       fips    0

In [39]:
# Create an instance of the RandomForestRegressor model.
rf = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=4)

# Train the RandomForestRegressor model on the training data (X_train_s, y_train).
# Evaluate its performance using the validation data (X_val_s, y_val).
train_model(rf, X_train_s, y_train, X_val_s, y_val)

The train RMSE is 267594.7713337082.
The validate RMSE is 364302.7418023686.


RandomForestRegressor(max_depth=10, min_samples_split=4)

## XGBRegressor

In [43]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [44]:
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

In [45]:
grid_search

GridSearchCV(cv=5,
             estimator=RandomForestRegressor(max_depth=10, min_samples_split=4),
             n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [46]:
train_model(rf, X_train_s, y_train, X_val_s, y_val)

The train RMSE is 265695.4384148166.
The validate RMSE is 366905.15756065975.


RandomForestRegressor(max_depth=10, min_samples_split=4)

In [49]:
grid_search.fit(X_train, y_train)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
148 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/ana

GridSearchCV(cv=5,
             estimator=RandomForestRegressor(max_depth=10, min_samples_split=4),
             n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [50]:
best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_

In [47]:
warnings.filterwarnings("ignore", category=FutureWarning, module="xgboost")
# Create an instance of the XGBoost Regressor model.
xgbr = XGBRegressor()

# Train the XGBoost Regressor model on the training data (X_train_s, y_train).
# Evaluate its performance using the validation data (X_val_s, y_val).
train_model(xgbr, X_train_s, y_train, X_val_s, y_val)

The train RMSE is 194472.201581023.
The validate RMSE is 368231.117766504.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [48]:
warnings.filterwarnings("ignore", category=FutureWarning, module="xgboost")
# Create an instance of the XGBoost Regressor model.
xgbr = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, subsample=1)

# Train the XGBoost Regressor model on the training data (X_train_s, y_train).
# Evaluate its performance using the validation data (X_val_s, y_val).
train_model(xgbr, X_train_s, y_train, X_val_s, y_val)

The train RMSE is 275258.3712898204.
The validate RMSE is 357228.542145154.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

df.latitude = df.latitude.astype(int)
df.longitude = df.longitude.astype(int)

# Assuming you have a DataFrame 'df' with 'latitude' and 'longitude' columns
latitude_df = pd.DataFrame(df['latitude'])
longitude_df = pd.DataFrame(df['longitude'])

# Perform reverse geocoding for each row in the DataFrames
for index, row in latitude_df.iterrows():
    latitude = row['latitude']
    longitude = longitude_df.at[index, 'longitude']  # Access longitude value using 'at'
    
    location = geocoder.osm([latitude, longitude], method='reverse')
    city = location.city
    df['city']= city
    print(f"City at index {index}: {city}")


pd.DataFrame(df.latitude).head()
pd.DataFrame(df.longitude).head()

# Assuming 'df' is your DataFrame with 'latitude' and 'longitude' columns
df['latitude'] = df['latitude'] / 1000000  # Add decimal place after the first two digits
df['longitude'] = df['longitude'] / 1000000  # Add decimal place after the first three digits

df.fips.value_counts()